In [2]:
! git clone https://github.com/NVlabs/stylegan
%cd stylegan/

Cloning into 'stylegan'...
remote: Enumerating objects: 83, done.
remote: Total 83 (delta 0), reused 0 (delta 0), pack-reused 83
Unpacking objects: 100% (83/83), done.
/content/stylegan


In [0]:
import os
import pickle
import numpy as np
import PIL.Image
import dnnlib
import dnnlib.tflib as tflib
import config

#----------------------------------------------------------------------------
# Helpers for loading and using pre-trained generators.

url_ffhq        = 'https://drive.google.com/uc?id=1MEGjdvVpUsu1jB4zrXZN7Y4kBBOzizDQ' # karras2019stylegan-ffhq-1024x1024.pkl
url_celebahq    = 'https://drive.google.com/uc?id=1MGqJl28pN4t7SAtSrPdSRJSQJqahkzUf' # karras2019stylegan-celebahq-1024x1024.pkl
url_bedrooms    = 'https://drive.google.com/uc?id=1MOSKeGF0FJcivpBI7s63V9YHloUTORiF' # karras2019stylegan-bedrooms-256x256.pkl
url_cars        = 'https://drive.google.com/uc?id=1MJ6iCfNtMIRicihwRorsM3b7mmtmK9c3' # karras2019stylegan-cars-512x384.pkl
url_cats        = 'https://drive.google.com/uc?id=1MQywl0FNt6lHu8E_EUqnRbviagS7fbiJ' # karras2019stylegan-cats-256x256.pkl

synthesis_kwargs = dict(output_transform=dict(func=tflib.convert_images_to_uint8, nchw_to_nhwc=True), minibatch_size=8)

_Gs_cache = dict()

def load_Gs(url):
    if url not in _Gs_cache:
        with dnnlib.util.open_url(url, cache_dir=config.cache_dir) as f:
            _G, _D, Gs = pickle.load(f)
        _Gs_cache[url] = Gs
    return _Gs_cache[url]

# Download to Drive (works for big files)
# This only needs to be done once in a notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once in a notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [10]:
# Style mixing - mix n_mix_layer of mix n_mix_layer of mixing image to reference image.
from itertools import chain
import timeit
start = timeit.timeit()

tflib.init_tf()
os.makedirs(config.result_dir, exist_ok=True)

Gs = load_Gs(url_ffhq)
w = 1024
h = 1024
style_ranges = [range(8,18), range(2,4)]
master_seeds = [701,687,615,2268,888,829,1898,1733,1614,845,1967,1555,1157,1012,91,388,201,500]
mix_seeds = master_seeds 
original_seeds = master_seeds

mix_latents = np.stack(np.random.RandomState(seed).randn(Gs.input_shape[1]) for seed in mix_seeds)
original_latents = np.stack(np.random.RandomState(seed).randn(Gs.input_shape[1]) for seed in original_seeds)
mix_dlatents = Gs.components.mapping.run(mix_latents, None) # [seed, layer, component]
original_dlatents = Gs.components.mapping.run(original_latents, None) # [seed, layer, component]
mix_images = Gs.components.synthesis.run(mix_dlatents, randomize_noise=False, **synthesis_kwargs)
original_images = Gs.components.synthesis.run(original_dlatents, randomize_noise=False, **synthesis_kwargs)



for original_ind, original_image in enumerate(list(original_images)):
    # Save original image
    canvas_original = PIL.Image.new(mode = 'RGB', size = (w, h), color = 'white')
    canvas_original.paste(PIL.Image.fromarray(original_image, 'RGB'), (0, 0))
    file_name = "{}_original.png".format(original_seeds[original_ind])
    canvas_original.save(file_name)
    uploaded = drive.CreateFile({'title': file_name})
    uploaded.SetContentFile(file_name)
    uploaded.Upload()

    for style_ind, style in enumerate(list(style_ranges)):
        # Mix the images
        row_dlatents = np.stack([original_dlatents[original_ind]] * len(mix_seeds))
        row_dlatents[:, style] = mix_dlatents[:, style]
        row_images = Gs.components.synthesis.run(row_dlatents, randomize_noise=False, **synthesis_kwargs)

        # Save the mix images
        for variant, image in enumerate(list(row_images)):
            canvas_mix = PIL.Image.new('RGB', (w, h), 'white')
            canvas_mix.paste(PIL.Image.fromarray(image, 'RGB'), (0, 0))
            file_name = "{}_{}_{}.png".format(original_seeds[original_ind], style, mix_seeds[variant])
            canvas_mix.save(file_name)
            uploaded = drive.CreateFile({'title': file_name})
            uploaded.SetContentFile(file_name)
            uploaded.Upload()

end = timeit.timeit()
print(end - start)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2822: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  if self.run_code(code, result):


In [5]:
original_seeds

[[845]]